---
# ***🫀 Heart Disease Risk Prediction***

<img src="https://hinduja-prod-assets.s3.ap-south-1.amazonaws.com/s3fs-public/2024-03/Heart%20Failure%20and%20Symptoms.jpg" alt="Heart Disease" width="100%" style="border-radius:12px; margin-bottom:10px;">

-----


---

# 📌**Introduction**
Heart disease is one of the **leading causes of death worldwide**.  
Early detection and risk prediction can help doctors take **preventive actions** and save lives.  
Using the **Cleveland Heart Disease dataset**, we aim to apply **machine learning** to identify patients at **high risk**.  

---

## ⚠️ **Problem Statement**  
Our healthcare team wants to shift toward **preventive care**.  
The challenge is to build an **ML-based tool** that classifies individuals as **high risk** or **low risk**,  
so clinicians can prioritize **early intervention and monitoring**.  

---

## 🎯 **Goal**  
The goal of this project is to:  
✅ Clean and prepare the dataset.  
✅ Perform **EDA** to discover key risk factors.  
✅ Train and evaluate classification models.  
✅ Deploy a simple **web app** where users can input patient details and receive a real-time risk prediction.  

---


----
# ***Let's Start***
-----

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import RocCurveDisplay


In [2]:
#Loading the data
df = pd.read_csv('./data/Heart_disease_cleveland_new.csv')

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,0,145,233,1,2,150,0,2.3,2,0,2,0
1,67,1,3,160,286,0,2,108,1,1.5,1,3,1,1
2,67,1,3,120,229,0,2,129,1,2.6,1,2,3,1
3,37,1,2,130,250,0,0,187,0,3.5,2,0,1,0
4,41,0,1,130,204,0,2,172,0,1.4,0,0,1,0


In [4]:
df.shape

(303, 14)

---------
<h2 align="left"><font color=red>Dataset Description:</font></h2>
    
| __Variable__ | __Description__ |
|     :---      |       :---      |      
| __age__ | Age of the patient in years |
| __sex__ | Gender of the patient (0 = male, 1 = female) |
| __cp__ | Chest pain type: <br> 0: Typical angina <br> 1: Atypical angina <br> 2: Non-anginal pain <br> 3: Asymptomatic |
| __trestbps__ | Resting blood pressure in mm Hg |
| __chol__ | Serum cholesterol in mg/dl |                     
| __fbs__ | Fasting blood sugar level, categorized as above 120 mg/dl (1 = true, 0 = false) |
| __restecg__ | Resting electrocardiographic results: <br> 0: Normal <br> 1: Having ST-T wave abnormality <br> 2: Showing probable or definite left ventricular hypertrophy |  
| __thalach__ | Maximum heart rate achieved during a stress test |                      
| __exang__ | Exercise-induced angina (1 = yes, 0 = no) |
| __oldpeak__ | ST depression induced by exercise relative to rest |
| __slope__ | Slope of the peak exercise ST segment: <br> 0: Upsloping <br> 1: Flat <br> 2: Downsloping |                      
| __ca__ | Number of major vessels (0-4) colored by fluoroscopy |              
| __thal__ | Thalium stress test result: <br> 0: Normal <br> 1: Fixed defect <br> 2: Reversible defect <br> 3: Not described  |
| __target__ | Heart disease status (0 = no disease, 1 = presence of disease) |


-------------

In [5]:
#infromation of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


------
<h2 align="left"><font color=red>Inferences:</font></h2>

* __Number of Entries__: The dataset consists of __303 entries__, ranging from index 0 to 302.
    
    
* __Columns__: There are __14 columns__ in the dataset corresponding to various attributes of the patients and results of tests.
    
    
* __Data Types__:
    - Most of the columns (13 out of 14) are of the __int64__ data type.
    - Only the oldpeak column is of the float64 data type.
    
    
* __Missing Values__: There don't appear to be any missing values in the dataset as each column has 303 non-null entries.
-----

-----
<h2 align="left"><font color=red>Note:</font></h2>

Based on the data types and the feature explanations we had earlier, we can see that __9 columns__ (`sex`, `cp`, `fbs`, `restecg`, `exang`, `slope`, `ca`, `thal`, and `target`) are indeed __numerical__ in terms of data type, but __categorical__ in terms of their semantics. These features should be converted to string (__object__) data type for proper analysis and interpretation:

-----

In [6]:
# Define the continuous features
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Identify the features to be converted to object data type
features_to_convert = [feature for feature in df.columns if feature not in continuous_features]

# Convert the identified features to object data type
df[features_to_convert] = df[features_to_convert].astype('object')

df.dtypes

age           int64
sex          object
cp           object
trestbps      int64
chol          int64
fbs          object
restecg      object
thalach       int64
exang        object
oldpeak     float64
slope        object
ca           object
thal         object
target       object
dtype: object

In [7]:
#Summary statistics for numerical variables
df.describe().T


,count,mean,std,min,25%,50%,75%,max
age,303.0,54.438944,9.038662,29.0,48.0,56.0,61.0,77.0
trestbps,303.0,131.689769,17.599748,94.0,120.0,130.0,140.0,200.0
chol,303.0,246.693069,51.776918,126.0,211.0,241.0,275.0,564.0
thalach,303.0,149.607261,22.875003,71.0,133.5,153.0,166.0,202.0
oldpeak,303.0,1.039604,1.161075,0.0,0.0,0.8,1.6,6.2


------
<h3 align="left"><font color=red>Numerical Features:</font></h3>

* __`age`__: The average age of the patients is approximately 54.4 years, with the youngest being 29 and the oldest 77 years.
* __`trestbps`__: The average resting blood pressure is about 131.62 mm Hg, ranging from 94 to 200 mm Hg.
* __`chol`__: The average cholesterol level is approximately 246.26 mg/dl, with a minimum of 126 and a maximum of 564 mg/dl.
* __`thalach`__: The average maximum heart rate achieved is around 149.65, with a range from 71 to 202.
* __`oldpeak`__: The average ST depression induced by exercise relative to rest is about 1.04, with values ranging from 0 to 6.2.

-----

-----
Afterward, let's look at the summary statistics of the ***categorical features***:

-----

In [8]:
# Get the summary statistics for categorical variables
df.describe(include='object')

,sex,cp,fbs,restecg,exang,slope,ca,thal,target
count,303,303,303,303,303,303,303,303,303
unique,2,4,2,3,2,3,4,3,2
top,1,3,0,0,0,0,0,1,0
freq,206,144,258,151,204,142,180,168,164


------
<h3 align="left"><font color=red>Categorical Features (object data type):</font></h3>

* __`sex`__: There are two unique values, with males (denoted as 0) being the most frequent category, occurring 207 times out of 303 entries.
* __`cp`__: Four unique types of chest pain are present. The most common type is "__0__", occurring 143 times.
* __`fbs`__: There are two categories, and the most frequent one is "__0__" (indicating fasting blood sugar less than 120 mg/dl), which appears 258 times.
* __`restecg`__: Three unique results are present. The most common result is "__1__", appearing 152 times.
* __`exang`__: There are two unique values. The most frequent value is "__0__" (indicating no exercise-induced angina), which is observed 204 times.
* __`slope`__: Three unique slopes are present. The most frequent slope type is "__2__", which occurs 142 times.
* __`ca`__: There are five unique values for the number of major vessels colored by fluoroscopy, with "__0__" being the most frequent, occurring 175 times.
* __`thal`__: Four unique results are available. The most common type is "__2__" (indicating a reversible defect), observed 166 times.
* __`target`__: Two unique values indicate the presence or absence of heart disease. The value "__1__" (indicating the presence of heart disease) is the most frequent, observed in 165 entries.

-----

In [9]:
#lets check for duplicates
df.duplicated().sum()

0

----
*As you can see we also dont have any **duplicates**...So, we'll keep moving*

---

---
<p align="left"><font color=red>Now,</font></>I am going to check for outliers using the __IQR method__ for the continuous features:

----

In [10]:
continuous_features

['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

In [11]:
Q1 = df[continuous_features].quantile(0.25)
Q3 = df[continuous_features].quantile(0.75)
IQR = Q3 - Q1
outliers_count_specified = ((df[continuous_features] < (Q1 - 1.5 * IQR)) | (df[continuous_features] > (Q3 + 1.5 * IQR))).sum()

outliers_count_specified

age         0
trestbps    9
chol        5
thalach     1
oldpeak     5
dtype: int64

-----    
Upon identifying outliers for the specified continuous features, we found the following:

* __`trestbps`__: 9 outliers
* __`chol`__: 5 outliers
* __`thalach`__: 1 outlier
* __`oldpeak`__: 5 outliers
* __`age`__: No outliers

-----

------
<h3 align="left"><font color=red>One-hot Encoding Decision:</font></h3>
    
Based on the feature descriptions, let's decide on one-hot encoding:

1. __Nominal Variables__: These are variables with no inherent order. They should be one-hot encoded because using them as numbers might introduce an unintended order to the model.

2. __Ordinal Variables__: These variables have an inherent order. They don't necessarily need to be one-hot encoded since their order can provide meaningful information to the model.

Given the above explanation:

- __`sex`__: This is a binary variable with two categories (male and female), so it doesn't need one-hot encoding.

    
- __`cp`__: Chest pain type can be considered as nominal because there's no clear ordinal relationship among the different types of chest pain (like Typical angina, Atypical angina, etc.). It should be one-hot encoded.
  
    
- __`fbs`__: This is a binary variable (true or false), so it doesn't need one-hot encoding.

    
- __`restecg`__: This variable represents the resting electrocardiographic results. The results, such as "Normal", "Having ST-T wave abnormality", and "Showing probable or definite left ventricular hypertrophy", don't seem to have an ordinal relationship. Therefore, it should be one-hot encoded.

    
- __`exang`__: This is a binary variable (yes or no), so it doesn't need one-hot encoding.

    
- __`slope`__: This represents the slope of the peak exercise ST segment. Given the descriptions (Upsloping, Flat, Downsloping), it seems to have an ordinal nature, suggesting a particular order. Therefore, it doesn't need to be one-hot encoded.

    
- __`ca`__: This represents the number of major vessels colored by fluoroscopy. As it indicates a count, it has an inherent ordinal relationship. Therefore, it doesn't need to be one-hot encoded.

    
- __`thal`__: This variable represents the result of a thalium stress test. The different states, like "Normal", "Fixed defect", and "Reversible defect", suggest a nominal nature. Thus, it should be one-hot encoded.

<h4 align="left">Summary:</h4>

- __Need One-Hot Encoding__: __`cp`__, __`restecg`__, __`thal`__
- __Don't Need One-Hot Encoding__: __`sex`__, __`fbs`__, __`exang`__, __`slope`__, __`ca`__


------

In [12]:
# Implementing one-hot encoding on the specified categorical features
df_encoded = pd.get_dummies(df, columns=['cp', 'restecg', 'thal'], drop_first=True, dtype=int)

# Convert the rest of the categorical variables that don't need one-hot encoding to integer data type
features_to_convert = ['sex', 'fbs', 'exang', 'slope', 'ca', 'target']
for feature in features_to_convert:
    df_encoded[feature] = df_encoded[feature].astype(int)

df_encoded.dtypes

age            int64
sex            int32
trestbps       int64
chol           int64
fbs            int32
thalach        int64
exang          int32
oldpeak      float64
slope          int32
ca             int32
target         int32
cp_1           int32
cp_2           int32
cp_3           int32
restecg_1      int32
restecg_2      int32
thal_2         int32
thal_3         int32
dtype: object

In [13]:
# Displaying the resulting DataFrame after one-hot encoding
df_encoded.head()

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,slope,ca,target,cp_1,cp_2,cp_3,restecg_1,restecg_2,thal_2,thal_3
0,63,1,145,233,1,150,0,2.3,2,0,0,0,0,0,0,1,1,0
1,67,1,160,286,0,108,1,1.5,1,3,1,0,0,1,0,1,0,0
2,67,1,120,229,0,129,1,2.6,1,2,1,0,0,1,0,1,0,1
3,37,1,130,250,0,187,0,3.5,2,0,0,0,1,0,0,0,0,0
4,41,0,130,204,0,172,0,1.4,0,0,0,1,0,0,0,1,0,0


In [14]:
df_encoded.to_csv("./data/clean_data.csv", index=False)